# Preprocesamiento Mineria de Datos


*   Estudiante: Inca Cruz Carlos Eduardo

# Setting up PySpark 3.0 on Google Colab

In [1]:
#get package
!wget https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar -xvzf spark-3.2.0-bin-hadoop2.7.tgz
!pip install findspark
#env vars
import os
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"
#start spark
import findspark
findspark.init()
#import spark session
from pyspark.sql import SparkSession
#build app
spark = SparkSession.builder.appName("PySpark").getOrCreate()
print(spark.sparkContext.appName)
#create sparkcontext
sc=spark.sparkContext
#clear output
from google.colab import output
output.clear()

1. Similitud de Cosenos

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
def DistanciaCoseno(List1,List2):
    return dot(List1, List2)/(norm(List1)*norm(List2))


In [ ]:
# Inicializamos el RDD
numPointsRDD = sc.parallelize([[0.3, 0.8,0.5,0.2],[0.4, 0.4,0.5,0.2],[0.6, 0.1,0.9,0.2]])
# Obtenemos el producto cartesiano
rdd=numPointsRDD.cartesian(numPointsRDD)
# Obtenemos la distancia coseno para cada par de puntos
distRDD = rdd.map(lambda x: (x[0],x[1],str(DistanciaCoseno(x[0],x[1]))))
# Mostramos resultados
distRDD.toDF(["Punto 1","Punto 2","Distancia Coseno"]).show()
statRDD = distRDD.map(lambda x: float(x[2]))

minv, maxv, meanv = statRDD.min(), statRDD.max(), statRDD.mean()
print ("min:",minv,",max: ", maxv,",mean: ", meanv)

+--------------------+--------------------+------------------+
|             Punto 1|             Punto 2|  Distancia Coseno|
+--------------------+--------------------+------------------+
|[0.3, 0.8, 0.5, 0.2]|[0.3, 0.8, 0.5, 0.2]|               1.0|
|[0.3, 0.8, 0.5, 0.2]|[0.4, 0.4, 0.5, 0.2]|0.9254604351544817|
|[0.3, 0.8, 0.5, 0.2]|[0.6, 0.1, 0.9, 0.2]|0.6723280987166992|
|[0.4, 0.4, 0.5, 0.2]|[0.3, 0.8, 0.5, 0.2]|0.9254604351544817|
|[0.6, 0.1, 0.9, 0.2]|[0.3, 0.8, 0.5, 0.2]|0.6723280987166992|
|[0.4, 0.4, 0.5, 0.2]|[0.4, 0.4, 0.5, 0.2]|               1.0|
|[0.4, 0.4, 0.5, 0.2]|[0.6, 0.1, 0.9, 0.2]|0.8925774123174453|
|[0.6, 0.1, 0.9, 0.2]|[0.4, 0.4, 0.5, 0.2]|0.8925774123174453|
|[0.6, 0.1, 0.9, 0.2]|[0.6, 0.1, 0.9, 0.2]|               1.0|
+--------------------+--------------------+------------------+

min: 0.6723280987166992 ,max:  1.0 ,mean:  0.8867479880419169


Binarizacion

In [4]:
import numpy as np
# Crear rdd categorico
catPointsRDD = sc.parallelize(enumerate([['alto', 'caro', 'azul'],
                             ['medio', 'caro', 'verde'],
                             ['alto', 'barato', 'azul'],
                             ['medio', 'caro', 'vermelho'],
                             ['baixo', 'barato', 'verde'],
                            ]))
# obtener claves unicas
clavesRDD = (catPointsRDD
             .flatMap(lambda x: [((x[0],xi),1) for xi in x[1]])
             .reduceByKey(lambda x,y: x)
             .map(lambda x: x[0])
             )

claves = dict((v,k) for k,v in clavesRDD.collect())
nclaves = len(claves)
print (claves, nclaves)
def CreateNP(atributos,claves):  
    #Binarizar utilizando diccionario de claves
    
    array = np.zeros(len(claves))
    for atr in atributos:
        array[ claves[atr] ] = 1
    return array

# Converte o RDD para o formato binário, utilizando o dict chaves
binRDD = catPointsRDD.map(lambda rec: (rec[0],CreateNP(rec[1], claves)))
binRDD.collect()

{'alto': 2, 'caro': 0, 'baixo': 4, 'verde': 1, 'azul': 2, 'medio': 3, 'barato': 4, 'vermelho': 3} 8


[(0, array([1., 0., 1., 0., 0., 0., 0., 0.])),
 (1, array([1., 1., 0., 1., 0., 0., 0., 0.])),
 (2, array([0., 0., 1., 0., 1., 0., 0., 0.])),
 (3, array([1., 0., 0., 1., 0., 0., 0., 0.])),
 (4, array([0., 1., 0., 0., 1., 0., 0., 0.]))]

2.Normalizacion

In [ ]:
import math
def nor(x):
    cociente=math.sqrt(sum([i**2 for i in x]))
    return [i/cociente for i in x]



In [ ]:
numPointsRDD = sc.parallelize([[0.37454012, 0.95071431,7,77,0.73199394, 0.59865848,22,-222,0.73199394, 0.59865848,22,-222],[0.37454012, 0.95071431,7,77,0.73199394, 0.59865848,22,-222,0.73199394, 0.59865848,22,-222]])
distRDD = numPointsRDD.map(lambda x: (x,nor(x)))
print(distRDD.collect())
#Mostrar Dataset
distRDD.toDF(["Punto","Punto Normalizado"]).show()

[([0.37454012, 0.95071431, 7, 77, 0.73199394, 0.59865848, 22, -222, 0.73199394, 0.59865848, 22, -222], [0.0011530216774382237, 0.0029267737952364714, 0.021549498467794494, 0.23704448314573945, 0.002253443184066408, 0.0018429699996417401, 0.06772699518449699, -0.6834269514071969, 0.002253443184066408, 0.0018429699996417401, 0.06772699518449699, -0.6834269514071969]), ([0.37454012, 0.95071431, 7, 77, 0.73199394, 0.59865848, 22, -222, 0.73199394, 0.59865848, 22, -222], [0.0011530216774382237, 0.0029267737952364714, 0.021549498467794494, 0.23704448314573945, 0.002253443184066408, 0.0018429699996417401, 0.06772699518449699, -0.6834269514071969, 0.002253443184066408, 0.0018429699996417401, 0.06772699518449699, -0.6834269514071969])]
+--------------------+--------------------+
|               Punto|   Punto Normalizado|
+--------------------+--------------------+
|[0.37454012, 0.95...|[0.00115302167743...|
|[0.37454012, 0.95...|[0.00115302167743...|
+--------------------+--------------------+

3.N-grams

In [9]:
#Codigo en python
def obtenNGramas(sentence, n):
    l1=[x for x in sentence.split(" ")]
    return [[x for x in sentence.split(" ")][i:i+3] for i in range(len([x for x in sentence.split(" ")])-(3-1))]
# Prueba
listaPalabras='this is a foo bar sentences and i want to ngramize it'
obtenNGramas(listaPalabras, 3)

[['this', 'is', 'a'],
 ['is', 'a', 'foo'],
 ['a', 'foo', 'bar'],
 ['foo', 'bar', 'sentences'],
 ['bar', 'sentences', 'and'],
 ['sentences', 'and', 'i'],
 ['and', 'i', 'want'],
 ['i', 'want', 'to'],
 ['want', 'to', 'ngramize'],
 ['to', 'ngramize', 'it']]

In [ ]:
dataRDD=sc.parallelize(['this is a foo bar sentences and i want to ngramize it','this is a foo bar sentences and i want to ngramize it','this is a foo bar sentences and i want to ngramize it'],8)
dataRDD2 = dataRDD.map(lambda x: (x,str(obtenNGramas(x,3))))
print(dataRDD2.collect())


[('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]"), ('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]"), ('this is a foo bar sentences and i want to ngramize it', "[['this', 'is', 'a'], ['is', 'a', 'foo'], ['a', 'foo', 'bar'], ['foo', 'bar', 'sentences'], ['bar', 'sentences', 'and'], ['sentences', 'and', 'i'], ['and', 'i', 'want'], ['i', 'want', 'to'], ['want', 'to', 'ngramize'], ['to', 'ngramize', 'it']]")]


4.Escalonamiento

In [ ]:
def Escalonamiento(x):
    return [(i-min(x))/(max(x)-min(x)) for i in x]

print(Escalonamiento([10,11,12]))
#scale xi = (xi - minimum x) / (maximum x - minimum x)

dataRDD=sc.parallelize([[1,2,3],[4,5,6],[7,8,9],[10,11,12]],2)
dataRDD2 = dataRDD.map(lambda x: (str(x),str(Escalonamiento(x))))
print(dataRDD2.collect())
#Mostrar Dataset
dataRDD2.toDF(["Punto","Punto Escalonado"]).show()

[0.0, 0.5, 1.0]
[('[1, 2, 3]', '[0.0, 0.5, 1.0]'), ('[4, 5, 6]', '[0.0, 0.5, 1.0]'), ('[7, 8, 9]', '[0.0, 0.5, 1.0]'), ('[10, 11, 12]', '[0.0, 0.5, 1.0]')]
+------------+----------------+
|       Punto|Punto Escalonado|
+------------+----------------+
|   [1, 2, 3]| [0.0, 0.5, 1.0]|
|   [4, 5, 6]| [0.0, 0.5, 1.0]|
|   [7, 8, 9]| [0.0, 0.5, 1.0]|
|[10, 11, 12]| [0.0, 0.5, 1.0]|
+------------+----------------+



In [5]:
def Escalonamiento(x):
    return [(i-min(x))/(max(x)-min(x)) for i in x]

print(Escalonamiento([10,11,12]))
#scale xi = (xi - minimum x) / (maximum x - minimum x)

dataRDD=sc.parallelize([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[1,2,3],[4,5,6],[7,8,9],[10,11,12],[1,2,3],[4,5,6],[7,8,9],[10,11,12],[1,2,3],[4,5,6],[7,8,9],[10,11,12]],2)
dataRDD2 = dataRDD.map(lambda x: (str(x),str(Escalonamiento(x))))
print(dataRDD2.collect())
#Mostrar Dataset
dataRDD2.toDF(["Punto","Punto Escalonado"]).show()

[0.0, 0.5, 1.0]
[('[1, 2, 3]', '[0.0, 0.5, 1.0]'), ('[4, 5, 6]', '[0.0, 0.5, 1.0]'), ('[7, 8, 9]', '[0.0, 0.5, 1.0]'), ('[10, 11, 12]', '[0.0, 0.5, 1.0]'), ('[1, 2, 3]', '[0.0, 0.5, 1.0]'), ('[4, 5, 6]', '[0.0, 0.5, 1.0]'), ('[7, 8, 9]', '[0.0, 0.5, 1.0]'), ('[10, 11, 12]', '[0.0, 0.5, 1.0]'), ('[1, 2, 3]', '[0.0, 0.5, 1.0]'), ('[4, 5, 6]', '[0.0, 0.5, 1.0]'), ('[7, 8, 9]', '[0.0, 0.5, 1.0]'), ('[10, 11, 12]', '[0.0, 0.5, 1.0]'), ('[1, 2, 3]', '[0.0, 0.5, 1.0]'), ('[4, 5, 6]', '[0.0, 0.5, 1.0]'), ('[7, 8, 9]', '[0.0, 0.5, 1.0]'), ('[10, 11, 12]', '[0.0, 0.5, 1.0]')]
+------------+----------------+
|       Punto|Punto Escalonado|
+------------+----------------+
|   [1, 2, 3]| [0.0, 0.5, 1.0]|
|   [4, 5, 6]| [0.0, 0.5, 1.0]|
|   [7, 8, 9]| [0.0, 0.5, 1.0]|
|[10, 11, 12]| [0.0, 0.5, 1.0]|
|   [1, 2, 3]| [0.0, 0.5, 1.0]|
|   [4, 5, 6]| [0.0, 0.5, 1.0]|
|   [7, 8, 9]| [0.0, 0.5, 1.0]|
|[10, 11, 12]| [0.0, 0.5, 1.0]|
|   [1, 2, 3]| [0.0, 0.5, 1.0]|
|   [4, 5, 6]| [0.0, 0.5, 1.0]|
|   [7, 8, 

5. Estandarizacion

In [ ]:
import statistics as stats
def Estandarizacion(x):
    return [(i-(sum(x)/len(x)))/(stats.variance(x)) for i in x]

print(Estandarizacion([10,11,12]))
#scale xi = (xi - minimum x) / (maximum x - minimum x)

dataRDD=sc.parallelize([[1,2,3],[4,5,6],[7,8,9],[10,11,12]],2)
dataRDD2 = dataRDD.map(lambda x: (str(x),str(Estandarizacion(x))))
print(dataRDD2.collect())
dataRDD2.toDF(["Punto","Punto Estandarizado"]).show()

[-1.0, 0.0, 1.0]
[('[1, 2, 3]', '[-1.0, 0.0, 1.0]'), ('[4, 5, 6]', '[-1.0, 0.0, 1.0]'), ('[7, 8, 9]', '[-1.0, 0.0, 1.0]'), ('[10, 11, 12]', '[-1.0, 0.0, 1.0]')]
+------------+----------------+
|       Punto|Punto Escalonado|
+------------+----------------+
|   [1, 2, 3]|[-1.0, 0.0, 1.0]|
|   [4, 5, 6]|[-1.0, 0.0, 1.0]|
|   [7, 8, 9]|[-1.0, 0.0, 1.0]|
|[10, 11, 12]|[-1.0, 0.0, 1.0]|
+------------+----------------+



In [6]:
import statistics as stats
def Estandarizacion(x):
    return [(i-(sum(x)/len(x)))/(stats.variance(x)) for i in x]

print(Estandarizacion([10,11,12]))
#scale xi = (xi - minimum x) / (maximum x - minimum x)

dataRDD=sc.parallelize([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[1,2,3],[4,5,6],[7,8,9],[10,11,12],[1,2,3],[4,5,6],[7,8,9],[10,11,12],[1,2,3],[4,5,6],[7,8,9],[10,11,12]],2)

dataRDD2 = dataRDD.map(lambda x: (str(x),str(Estandarizacion(x))))
print(dataRDD2.collect())
dataRDD2.toDF(["Punto","Punto Estandarizado"]).show()


[-1.0, 0.0, 1.0]
[('[1, 2, 3]', '[-1.0, 0.0, 1.0]'), ('[4, 5, 6]', '[-1.0, 0.0, 1.0]'), ('[7, 8, 9]', '[-1.0, 0.0, 1.0]'), ('[10, 11, 12]', '[-1.0, 0.0, 1.0]'), ('[1, 2, 3]', '[-1.0, 0.0, 1.0]'), ('[4, 5, 6]', '[-1.0, 0.0, 1.0]'), ('[7, 8, 9]', '[-1.0, 0.0, 1.0]'), ('[10, 11, 12]', '[-1.0, 0.0, 1.0]'), ('[1, 2, 3]', '[-1.0, 0.0, 1.0]'), ('[4, 5, 6]', '[-1.0, 0.0, 1.0]'), ('[7, 8, 9]', '[-1.0, 0.0, 1.0]'), ('[10, 11, 12]', '[-1.0, 0.0, 1.0]'), ('[1, 2, 3]', '[-1.0, 0.0, 1.0]'), ('[4, 5, 6]', '[-1.0, 0.0, 1.0]'), ('[7, 8, 9]', '[-1.0, 0.0, 1.0]'), ('[10, 11, 12]', '[-1.0, 0.0, 1.0]')]
+------------+-------------------+
|       Punto|Punto Estandarizado|
+------------+-------------------+
|   [1, 2, 3]|   [-1.0, 0.0, 1.0]|
|   [4, 5, 6]|   [-1.0, 0.0, 1.0]|
|   [7, 8, 9]|   [-1.0, 0.0, 1.0]|
|[10, 11, 12]|   [-1.0, 0.0, 1.0]|
|   [1, 2, 3]|   [-1.0, 0.0, 1.0]|
|   [4, 5, 6]|   [-1.0, 0.0, 1.0]|
|   [7, 8, 9]|   [-1.0, 0.0, 1.0]|
|[10, 11, 12]|   [-1.0, 0.0, 1.0]|
|   [1, 2, 3]|   [-1.0, 0